In [1]:
%pwd


'/Users/rohithkamath/Documents/endtoendproject/Text_Summarization_NLP/trial'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rohithkamath/Documents/endtoendproject/Text_Summarization_NLP'

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path



In [32]:
from textsummarizer.constants import *
from textsummarizer.utils.common_func import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_url=config.source_url,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [33]:
from urllib import request
import zipfile
import os
from textsummarizer.logging import logger
from textsummarizer.utils.common_func import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading file from: {self.config.source_url}")
            request.urlretrieve(self.config.source_url, self.config.local_data_file)
        else:
            logger.info(f"File already exists at: {self.config.local_data_file}.")
            logger.info(f"File size: {get_size(self.config.local_data_file)} bytes.")

    def extract_zip_file(self):
        
        logger.info(f"Extracting file to: {self.config.unzip_dir}")
        create_directories([self.config.unzip_dir])
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_id:
            zip_id.extractall(self.config.unzip_dir)

In [34]:
try: 
    congiuration_manager = ConfigurationManager()
    data_ingestion_config = congiuration_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-17 09:59:04,212: INFO:  common_func:  Yaml file: config/config.yaml has been loaded succesfully]


[2025-05-17 09:59:04,222: INFO:  common_func:  Yaml file: params.yaml has been loaded succesfully]
[2025-05-17 09:59:04,225: INFO:  common_func:  created directory at: artifacts]
[2025-05-17 09:59:04,227: INFO:  common_func:  created directory at: artifacts/data_ingestion]
[2025-05-17 09:59:04,229: INFO:  2968198114:  Downloading file from: https://github.com/entbappy/Branching-tutorial/blob/master/summarizer-data.zip]
